In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import logging
import time
import pandas as pd
from scipy.ndimage import zoom
from giza_actions.action import action
from giza_actions.task import task
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Define the model


In [3]:
input_size = 99
output_size = 1

# Define hidden layers sizes and dropout rates
hidden_size = [256, 128, 64]
dropout_rates = [0.1, 0.1]
learning_rate = 0.001
num_epochs = 100
batch_size = 64

In [4]:
class RegressionNN(nn.Module):
    def __init__(self, input_size, hidden_size=[256, 128, 64], output_size=1, dropout_rates=[0.1, 0.1]):
        super(RegressionNN, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()

        layer_sizes = [input_size] + hidden_size
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            self.batch_norms.append(nn.BatchNorm1d(layer_sizes[i+1]))
            self.dropouts.append(nn.Dropout(dropout_rates[min(i, len(dropout_rates)-1)]))
        
        self.output_layer = nn.Linear(hidden_size[-1], output_size)
        
    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            x = self.batch_norms[i](x)
            x = nn.ReLU()(x)
            x = self.dropouts[i](x)
            
        x = self.output_layer(x)
        return x


In [5]:
@task(name=f'Prepare Datasets')
def prepare_datasets(X, y):
    print("Prepare dataset...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train_tensor = torch.tensor(X_train.values.astype(np.float32))
    y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).view(-1, 1)
    X_test_tensor = torch.tensor(X_test.values.astype(np.float32))
    y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).view(-1, 1)

    print("✅ Datasets prepared successfully")

    return X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor

In [6]:
@task(name=f'Create Loaders')
def create_data_loaders(x_train, y_train, x_test, y_test):
    print("Create loaders...")

    train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

    print("✅ Loaders created!")

    return train_loader, test_loader

In [7]:
@task(name=f'Train model')
def train_model(train_loader):
    print("Train model...")

    model = RegressionNN(input_size, hidden_size, output_size=1).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        start_time = time.time()
        train_loss = 0.0
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        
    print("✅ Model trained successfully")
    return model

In [8]:
@task(name=f'Test model')
def test_model(model, test_loader):
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predictions.extend(outputs.view(-1).cpu().numpy())  # Flatten outputs and move to CPU
            actuals.extend(targets.view(-1).cpu().numpy())  # Flatten targets and move to CPU

    predictions = np.array(predictions)
    actuals = np.array(actuals)

    # Calculate MAE
    mae = np.mean(np.abs(predictions - actuals))

    # Calculate MAPE
    mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100
    print('Mean absolute error:', mae)
    print('MAPE:', mape)
            

In [9]:
@action(name=f'Execution', log_prints=True )
def execution():
    #read csv
    sales = pd.read_csv('sales.csv')
    X = sales[pd.Series(sales.columns)[pd.Series(sales.columns)!='amount_original'].values]
    y = sales['amount_original']  # Your target vector
    x_train, y_train, x_test, y_test = prepare_datasets(X, y)
    train_loader, test_loader = create_data_loaders(
        x_train, y_train, x_test, y_test)
    model = train_model(train_loader)
    test_model(model, test_loader)


In [10]:
execution()

15:43:14.191 | INFO    | prefect.engine - Created flow run 'black-oriole' for flow 'Execution'

15:43:14.197 | INFO    | Flow run 'black-oriole' - View at https://actions-server-lucab-dblzzhtf5q-ew.a.run.app/flow-runs/flow-run/81c7c1cd-a27f-43a3-aa1d-131849a59597

15:43:15.536 | INFO    | Flow run 'black-oriole' - Created task run 'Prepare Datasets-0' for task 'Prepare Datasets'

15:43:15.539 | INFO    | Flow run 'black-oriole' - Executing 'Prepare Datasets-0' immediately...

15:43:16.077 | INFO    | Task run 'Prepare Datasets-0' - Prepare dataset...

...

15:45:03.040 | INFO    | Task run 'Test model-0' - Mean absolute error: 6.463292

15:45:03.044 | INFO    | Task run 'Test model-0' - MAPE: 16.46459996700287

15:45:03.145 | INFO    | Task run 'Test model-0' - Finished in state Completed()

15:45:03.271 | INFO    | Flow run 'black-oriole' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `RegressionNN`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]